In [1]:
import sys
sys.path.append(r'C:\Users\moallemie\EMAworkbench-master')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import load_results, ema_logging

In [4]:
sc = 3000    # Specify the number of scenarios where the convergence in the SA indices occured
n_exp = 2000  # Indicate the number of LHS experiments to run 
t = 2030
nprocess = 25

## Load  model, uncertainities; generate initial experiments

In [5]:
# Open Excel input data from the notebook directory before runnign the code in multi-processing.

def create_sets(outcome_var):
    
    from Model_init import vensimModel
    
    directory = 'D:/moallemie/GitHub/Scenario_Modelling/Model/'

    df_unc = pd.read_excel(directory+'ScenarioFramework.xlsx', sheet_name='Uncertainties')

    # 0.75/1.25 multiplication is added to Min/Max for parameters with Reference values 0 
    #or min/max manually set in the spreadsheet   
    #df_unc['Min'] = df_unc['Min'] + df_unc['Reference'] * 0.75
    #df_unc['Max'] = df_unc['Max'] + df_unc['Reference'] * 1.25



    # From the Scenario Framework (all uncertainties), filter only those top 20 sensitive uncertainties under each outcome
    sa_dir='D:/moallemie/GitHub/Scenario_Modelling/Results/Morris_results/'

    mu_df = pd.read_csv(sa_dir+"MorrisIndices_{}_sc{}_t{}_test.csv".format(outcome_var, sc, t))
    mu_df.rename(columns={'Unnamed: 0': 'Uncertainty'}, inplace=True)
    mu_df.sort_values(by=['mu_star'], ascending=False, inplace=True)
    mu_df = mu_df.head(20)
    mu_unc = mu_df['Uncertainty']
    mu_unc_df = mu_unc.to_frame()

    # Remove the rest of insensitive uncertainties from the Scenario Framework and update df_unc
    keys = list(mu_unc_df.columns.values)
    i1 = df_unc.set_index(keys).index
    i2 = mu_unc_df.set_index(keys).index
    df_unc2 = df_unc[i1.isin(i2)]


    # Reorder the dataframe of mu index according to the model uncertainty.
    # It will be used when we generate Sets 1 & 2 
    mu_df1 = mu_df[['Uncertainty', 'mu_star']]
    mu_df1 = mu_df1.set_index('Uncertainty')
    mu_df1 = mu_df1.reindex(index=df_unc2['Uncertainty'])
    mu_df1 = mu_df1.reset_index()


    vensimModel.uncertainties = [RealParameter(row['Uncertainty'], row['Min'], row['Max']) for index, row in df_unc2.iterrows()]

    df_out = pd.read_excel(directory+'ScenarioFramework.xlsx', sheet_name='Outcomes')
    vensimModel.outcomes = [TimeSeriesOutcome(out) for out in df_out['Outcome']]



    from ema_workbench import MultiprocessingEvaluator
    from ema_workbench.em_framework.evaluators import (MC, LHS, FAST, FF, PFF, SOBOL, MORRIS)

    try:
        with MultiprocessingEvaluator(vensimModel, n_processes=nprocess) as evaluator:
            results = evaluator.perform_experiments(scenarios=n_exp, uncertainty_sampling=LHS) # The number of scenarios here is only for identifying the number of important parameters in SA results.
    except (BrokenPipeError, IOError):
        pass

    experiments, outcomes = results


    # Sort factors by importance
    inds_mu = mu_df1['mu_star'].values
    factors_sorted = np.argsort(inds_mu)[::-1]

    # Set up DataFrame of default values to use for experiment
    nsamples = len(experiments.index)

    defaultvalues = df_unc2['Reference'].values
    X_defaults = np.tile(defaultvalues,(nsamples, 1))


    # Create Set 1 from experiments
    exp_T = experiments.drop(['scenario', 'policy', 'model'], axis=1).T.reindex(df_unc2['Uncertainty'])
    exp_ordered = exp_T.T
    X_Set1 = exp_ordered.values

    # Create initial Sets 2 and 3
    X_Set2 = np.copy(X_defaults)
    X_Set3 = np.copy(X_Set1)

    return df_unc2, factors_sorted, X_defaults, X_Set1, X_Set2, X_Set3, outcomes, mu_df
    
   

## Where to draw the line between important and not important?

In [5]:
'''
Modifed from Waterprogramming blog by Antonia Hadgimichael: https://github.com/antonia-had/SA_verification

The idea is that we create 2 additiopnal Sets (current SA samples are Set 1).

We create a Set 2, using only the T most important factors from our Set 1 sample, 
and fixing all other factors to their default values.

We also create a Set 3, now fixing the T most important factors to defaults 
and using the sampled values of all other factors from Set 1.

If we classified our important and unimportant factors correctly, 
then the correlation coefficient between the model outputs of Set 2 and Set 1 should approximate 1 
(since we’re fixing all factors that don’t matter), 
and the correlation coefficient between outputs from Set 3 and Set 1 should approximate 0 
(since the factors we sampled are inconsequential to the output).

'''

'\nModifed from Waterprogramming blog by Antonia Hadgimichael: https://github.com/antonia-had/SA_verification\n\nThe idea is that we create 2 additiopnal Sets (current SA samples are Set 1).\n\nWe create a Set 2, using only the T most important factors from our Set 1 sample, \nand fixing all other factors to their default values.\n\nWe also create a Set 3, now fixing the T most important factors to defaults \nand using the sampled values of all other factors from Set 1.\n\nIf we classified our important and unimportant factors correctly, \nthen the correlation coefficient between the model outputs of Set 2 and Set 1 should approximate 1 \n(since we’re fixing all factors that don’t matter), \nand the correlation coefficient between outputs from Set 3 and Set 1 should approximate 0 \n(since the factors we sampled are inconsequential to the output).\n\n'

In [6]:
# Define a function to convert your Set 2 and Set 3 into experiments structure in the EMA Workbench

def SA_experiments_to_scenarios(experiments, top_unc, model=None):
    '''

    "Slighlty modifed from the EMA Workbench"
    
    This function transform a structured experiments array into a list
    of Scenarios.

    If model is provided, the uncertainties of the model are used.
    Otherwise, it is assumed that all non-default columns are
    uncertainties.

    Parameters
    ----------
    experiments : numpy structured array
                  a structured array containing experiments
    model : ModelInstance, optional

    Returns
    -------
    a list of Scenarios

    '''
    from ema_workbench import Scenario
    
    # get the names of top 20 uncertainties
    uncertainties = list(top_unc['Uncertainty'])

    # make list of of tuples of tuples
    cases = []
    cache = set()
    for i in range(experiments.shape[0]):
        case = {}
        case_tuple = []
        for uncertainty in uncertainties:
            entry = experiments[uncertainty][i]
            case[uncertainty] = entry
            case_tuple.append(entry)

        case_tuple = tuple(case_tuple)
        cases.append(case)
        cache.add((case_tuple))

    scenarios = [Scenario(**entry) for entry in cases]

    return scenarios

# Run the models for the top n factors in Set 2 and Set 3 and generate correlation figures

def ranking_verification(outcome_var, df_unc2, factors_sorted, X_defaults, X_Set1, X_Set2, X_Set3, outcomes):
    
    from Model_init import vensimModel
    
    coefficient_S1_S2 = 0
    coefficient_S1_S3 = 0.99

    n = 0    
    for f in range(1, len(factors_sorted)+1):
        ntopfactors = f
        if (coefficient_S1_S2 <=0.95 or coefficient_S1_S3 >= 0.1):

            for i in range(ntopfactors): #Loop through all important factors
                X_Set2[:,factors_sorted[i]] = X_Set1[:,factors_sorted[i]] #Fix use samples for important
                X_Set3[:,factors_sorted[i]] = X_defaults[:,factors_sorted[i]] #Fix important to defaults

            X_Set2_exp = pd.DataFrame(data=X_Set2, columns=df_unc2['Uncertainty'].tolist())
            X_Set3_exp = pd.DataFrame(data=X_Set3, columns=df_unc2['Uncertainty'].tolist())

            scenarios_Set2 = SA_experiments_to_scenarios(X_Set2_exp, df_unc2, model=vensimModel)
            scenarios_Set3 = SA_experiments_to_scenarios(X_Set3_exp, df_unc2, model=vensimModel)
            
            vensimModel.outcomes = [TimeSeriesOutcome(outcome_var)]

            with MultiprocessingEvaluator(vensimModel, n_processes=nprocess) as evaluator:
                    experiments_Set2, outcomes_Set2 = evaluator.perform_experiments(scenarios=scenarios_Set2)
                    experiments_Set3, outcomes_Set3 = evaluator.perform_experiments(scenarios=scenarios_Set3)


            # Calculate coefficients of correlation
            data_Set1 = outcomes[outcome_var][:,-1]
            data_Set2 = outcomes_Set2[outcome_var][:,-1]
            data_Set3 = outcomes_Set3[outcome_var][:,-1]

            coefficient_S1_S2 = np.corrcoef(data_Set1,data_Set2)[0][1]
            coefficient_S1_S3 = np.corrcoef(data_Set1,data_Set3)[0][1]

            # Plot outputs and correlation
            fig =  plt.figure(figsize=(14,7))
            ax1 = fig.add_subplot(1,2,1)
            ax1.plot(data_Set1,data_Set1, color='#39566E')
            ax1.scatter(data_Set1,data_Set2, color='#8DCCFC')
            ax1.set_xlabel("Set 1",fontsize=14)
            ax1.set_ylabel("Set 2",fontsize=14)
            ax1.tick_params(axis='both', which='major', labelsize=10)
            ax1.set_title('Set 1 vs Set 2 - ' + str(f) + ' top factors',fontsize=15)
            ax1.text(0.05,0.95,'R= '+"{0:.3f}".format(coefficient_S1_S2),transform = ax1.transAxes,fontsize=16)
            ax2 = fig.add_subplot(1,2,2)
            ax2.plot(data_Set1,data_Set1, color='#39566E')
            ax2.scatter(data_Set1,data_Set3, color='#FFE0D5')
            ax2.set_xlabel("Set 1",fontsize=14)
            ax2.set_ylabel("Set 3",fontsize=14)
            ax2.tick_params(axis='both', which='major', labelsize=10)
            ax2.set_title('Set 1 vs Set 3 - ' + str(f) + ' top factors',fontsize=15)
            ax2.text(0.05,0.95,'R= '+"{0:.3f}".format(coefficient_S1_S3),transform = ax2.transAxes,fontsize=16)
            plt.savefig('{}/{}_{}_topfactors.png'.format(r'C:/Users/moallemie/EM_analysis/Fig/sa_verification', outcome_var, str(f)),bbox_inches='tight')
            plt.close()

            n += 1
    
    n_factor = n
    return fig, n_factor

In [7]:
# Run the models for the top n factors in Set 2 and Set 3 and generate correlation figures

def ranking_verification(outcome_var, df_unc2, factors_sorted, X_defaults, X_Set1, X_Set2, X_Set3, outcomes):
    
    from Model_init import vensimModel
    
    coefficient_S1_S2 = 0
    coefficient_S1_S3 = 0.99
    pd_corr = pd.DataFrame(columns=['Outcome', 'Number_of_top_parameters', 'Correlation_Set1_Set3', 'Correlation_Set1_Set2'])

    n = 0    
    for f in range(1, len(factors_sorted)+1):
        ntopfactors = f
        if (coefficient_S1_S2 <=0.99 or coefficient_S1_S3 >= 0.5):

            for i in range(ntopfactors): #Loop through all important factors
                X_Set2[:,factors_sorted[i]] = X_Set1[:,factors_sorted[i]] #Fix use samples for important
                X_Set3[:,factors_sorted[i]] = X_defaults[:,factors_sorted[i]] #Fix important to defaults

            X_Set2_exp = pd.DataFrame(data=X_Set2, columns=df_unc2['Uncertainty'].tolist())
            X_Set3_exp = pd.DataFrame(data=X_Set3, columns=df_unc2['Uncertainty'].tolist())

            scenarios_Set2 = SA_experiments_to_scenarios(X_Set2_exp, df_unc2, model=vensimModel)
            scenarios_Set3 = SA_experiments_to_scenarios(X_Set3_exp, df_unc2, model=vensimModel)
            
            vensimModel.outcomes = [TimeSeriesOutcome(outcome_var)]

            try:
                with MultiprocessingEvaluator(vensimModel, n_processes=nprocess) as evaluator:
                        experiments_Set2, outcomes_Set2 = evaluator.perform_experiments(scenarios=scenarios_Set2)
            except (BrokenPipeError, IOError):
                pass
            
            try:
                with MultiprocessingEvaluator(vensimModel, n_processes=nprocess) as evaluator:
                        experiments_Set3, outcomes_Set3 = evaluator.perform_experiments(scenarios=scenarios_Set3)
            except (BrokenPipeError, IOError):
                pass

            # Calculate coefficients of correlation
            data_Set1 = outcomes[outcome_var][:,-1]
            data_Set2 = outcomes_Set2[outcome_var][:,-1]
            data_Set3 = outcomes_Set3[outcome_var][:,-1]

            coefficient_S1_S2 = np.corrcoef(data_Set1,data_Set2)[0][1]
            coefficient_S1_S3 = np.corrcoef(data_Set1,data_Set3)[0][1]
            
            pd_corr.loc[n] = [outcome_var, ntopfactors, coefficient_S1_S3, coefficient_S1_S2]

            # Plot outputs and correlation
            fig =  plt.figure(figsize=(14,7))
            ax1 = fig.add_subplot(1,2,1)
            ax1.plot(data_Set1,data_Set1, color='#39566E')
            ax1.scatter(data_Set1,data_Set2, color='#8DCCFC')
            ax1.set_xlabel("Set 1",fontsize=14)
            ax1.set_ylabel("Set 2",fontsize=14)
            ax1.tick_params(axis='both', which='major', labelsize=10)
            ax1.set_title('Set 1 vs Set 2 - ' + str(f) + ' top factors',fontsize=15)
            ax1.text(0.05,0.95,'R= '+"{0:.3f}".format(coefficient_S1_S2),transform = ax1.transAxes,fontsize=16)
            ax2 = fig.add_subplot(1,2,2)
            ax2.plot(data_Set1,data_Set1, color='#39566E')
            ax2.scatter(data_Set1,data_Set3, color='#FFE0D5')
            ax2.set_xlabel("Set 1",fontsize=14)
            ax2.set_ylabel("Set 3",fontsize=14)
            ax2.tick_params(axis='both', which='major', labelsize=10)
            ax2.set_title('Set 1 vs Set 3 - ' + str(f) + ' top factors',fontsize=15)
            ax2.text(0.05,0.95,'R= '+"{0:.3f}".format(coefficient_S1_S3),transform = ax2.transAxes,fontsize=16)
            plt.savefig('{}/{}_{}_topfactors.png'.format(r'D:/moallemie/GitHub/Scenario_Modelling/Fig/sa_verification', outcome_var, str(f)),bbox_inches='tight')
            plt.close()

            n += 1
    
    n_factor = n
    return fig, n_factor, pd_corr

In [8]:
# define the ploting function
def plot_scores(outcome_var, n, sc, t):
    mu_df2 = mu_df.set_index('Uncertainty')
    mu_df2.sort_values(by=['mu_star'], ascending=True, inplace=True)

    sns.set_style('white')
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 6)) 
    ind = mu_df2.iloc[:,0]
    err = mu_df2.iloc[:,1]
    ind.plot.barh(xerr=err.values.T,ax=ax, color = ['#D7F0BC']*(20-n)+['#62C890']*n, 
                  ecolor='dimgray', capsize=2,  width=.9)
   
    ax.set_ylabel('')
    ax.legend().set_visible(False)
    ax.set_xlabel('mu_star index', fontsize=12)

    ylabels = ax.get_yticklabels()
    ylabels = [item.get_text()[:-10] for item in ylabels]
    ax.set_yticklabels(ylabels, fontsize=12)
    title = outcome_var[:-10]
    ax.set_title("{}".format(title), fontsize=12)
    plt.suptitle("Target year: "+str(t), y=0.94, fontsize=12)
    plt.rcParams["figure.figsize"] = [7.08,7.3]
    plt.savefig('{}/Morris_ranking_sorted_{}_sc{}_t{}.png'.format(r'D:/moallemie/GitHub/Scenario_Modelling/Fig/sa_verification', outcome_var, sc, t), dpi=300,  bbox_inches='tight')
    return fig

In [9]:
if __name__ == '__main__':
    
    ema_logging.log_to_stderr(ema_logging.INFO)

    from ema_workbench import (TimeSeriesOutcome, 
                                   perform_experiments,
                                   RealParameter, 
                                   CategoricalParameter,
                                   ema_logging, 
                                   save_results,
                                  load_results)

    
    from ema_workbench import MultiprocessingEvaluator

    out_dir = 'D:/moallemie/GitHub/Scenario_Modelling/Model/'
    df_out = pd.read_excel(out_dir+'ScenarioFramework.xlsx', sheet_name='Outcomes')
    outcome_vars = ['Total Population Indicator',
                     'Total Primary Education Graduates Indicator',
                     'Total Secondary Education Graduates Indicator',
                     'Total Tertiary Education Graduates Indicator',
                     'GWP per Capita Indicator',
                     'Energy Demand Indicator',
                     'Solar Energy Production Indicator',
                     'Wind Energy Production Indicator',
                     'Biomass Energy Production Indicator',
                     'Oil Production Indicator',
                     'Gas Production Indicator',
                     'Coal Production Indicator',
                     'Total Croplands Indicator',
                     'Forest Land Indicator',
                     'Pasture Land Indicator',
                     'Urban Industrial Land Indicator',
                     'Nonenergy Crops Production Indicator',
                     'Livestock Production Indicator',
                     'Total CO2 Emissions Indicator',
                     'CO2 Radiative Forcing Indicator']


    import  time
    start = time.time()

    top_factors = {}
    pd_corr = pd.DataFrame(columns=['Outcome', 'Number_of_top_parameters', 'Correlation_Set1_Set3', 'Correlation_Set1_Set2'])

    for out, outcome_var in enumerate(outcome_vars):
        ema_logging.log_to_stderr(ema_logging.INFO)
        df_unc2, factors_sorted, X_defaults, X_Set1, X_Set2, X_Set3, outcomes, mu_df = create_sets(outcome_var)
        fig, n_factor, pd_corr_var = ranking_verification(outcome_var, df_unc2, factors_sorted, X_defaults, X_Set1, X_Set2, X_Set3, outcomes)
        plot_scores(outcome_var, n_factor, sc, t)
        plt.close()

        top_factors[outcome_var] = n_factor
        pd_corr = pd.concat([pd_corr, pd_corr_var])
        
    import json
    with open('D:/moallemie/GitHub/Scenario_Modelling/Fig/sa_verification/note.txt', 'w') as file:
         file.write(json.dumps(top_factors))

    end = time.time()
    print("took {} seconds".format(end-start))

[MainProcess/INFO] using 64 bit vensim
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProc

[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProces

[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainPro

[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess

[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProce

...

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainPro

[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess

[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProce

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 1 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainPro

took 272334.51307344437 seconds


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd_corr)

,Outcome,Number_of_top_parameters,Correlation_Set1_Set3,Correlation_Set1_Set2
0,Total Population Indicator,1,0.796732,0.602060
1,Total Population Indicator,2,0.472579,0.886487
2,Total Population Indicator,3,0.465938,0.889934
3,Total Population Indicator,4,0.428209,0.907752
4,Total Population Indicator,5,0.421604,0.908302
5,Total Population Indicator,6,0.420702,0.908912
6,Total Population Indicator,7,0.420727,0.909021
7,Total Population Indicator,8,0.420779,0.909014
8,Total Population Indicator,9,0.391245,0.918922
9,Total Population Indicator,10,0.286047,0.956745
